In [14]:
from collections import namedtuple
import numpy as np
from random import random
from queue import PriorityQueue, SimpleQueue , LifoQueue
from functools import reduce

PROBLEM_SIZE = 10
NUM_SETS = 16

SETS = tuple([np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)])

State = namedtuple('State', ['taken', 'untaken'])

def goal_check(state):
    return np.all(reduce(np.logical_or,[SETS[i] for i in state.taken],  np.array([False for _ in range(PROBLEM_SIZE)]) ))

A state is the set of elements taken + the set of element not taken
one possible state => ({1,3,5} , {0,2,4,5,7})
sets[i] for i in state[0] are the taken elements in our sets
Then I need a way to test if the actual state is the goal one:
    looking at the taken elements and then checking if we have take all the elements with a bitwise OR
First of all we check if the problem is solvable

In [15]:
assert goal_check(State(set(range(NUM_SETS)),set())), "Goal check failed"

Then we need to define the frontier and put the initial state

In [18]:
frontier = SimpleQueue() ## Breadth first search
#frontier = LifoQueue() ## Depth first search
frontier.put(State(set(),set(range(NUM_SETS))))
current_state = frontier.get()
counter = 0
while not goal_check(current_state):
    counter += 1
    for action in current_state.untaken:
        new_state = State(current_state.taken.union({action}),current_state.untaken.difference({action}))
        frontier.put(new_state)
    current_state = frontier.get()
assert goal_check(current_state), "Goal check failed"
print(f"Found solution in {format(counter)} iterations,  steps : ({len(current_state.taken)})")

Found solution in 6567 iterations,  steps : (4)


Now write the REAL solution with working priority queue -> Dijkstra algorithm with smaller first

Following, greedy solution based on the distance from the goal state

In [22]:
def distance(state):
    return PROBLEM_SIZE - sum (
        reduce(np.logical_or,[SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]) ))

frontier = PriorityQueue()
state = State(set(),set(range(NUM_SETS)))
frontier.put((distance(state),state))
_ , current_state = frontier.get()
counter = 0
while not goal_check(current_state):
    counter += 1
    for action in current_state.untaken:
        new_state = State(current_state.taken.union({action}),current_state.untaken.difference({action}))
        frontier.put((distance(new_state),new_state))
    _ , current_state = frontier.get()
assert goal_check(current_state), "Goal check failed"
print(f"Found solution in {format(counter)} iterations,  steps : ({len(current_state.taken)})")

Found solution in 4 iterations,  steps : (4)
